Now that the main analysis has been run, let's create some final plots.

In [ ]:
import json
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from matplotlib.lines import Line2D
import numpy as np
import pandas as pd

capacities = {
    "Waterbuffel":103,
    "Watergeus":103,
    "Waterman":83,
    "Waterval":83,
    "Watervogel":140
}

In [ ]:
def location_in_area(location, area):
    (px, py) = location
    ((rx_min, ry_min), (rx_max, ry_max)) = area
    
    # Check if within boundaries
    if rx_min <= px <= rx_max and ry_min <= py <= ry_max:
        return 1

    else:
        return 0

### Q1

In [ ]:
# Import Evides & remove shipments from passagiersschepen, opleidingsschepen. Also remove shipments which are not in the general port area. 
evides = pd.read_csv('data/evides_withAIS.csv')
evides = evides[evides['Scheepstype'] != 'Passagiersschip, opleidingschip']
evides['Outside area'] = [location_in_area([evides['Latitude'].loc[x],evides['Longitude'].loc[x]], ((51.86, 3.9), (52,4.6))) for x in evides.index]
evides = evides[evides['Outside area']==1]

# Get original data
date = '2022-07-28'
data = {x:{'Cap':capacities[x], 
           'Delivered':sum(evides['Hoeveelheid (m3)'][(evides['Waterboot']==x)&(evides['Datum']==date)])} for x in set(evides['Waterboot'][evides['Datum']==date])}
for x in data:
    data[x]['Used_CAP'] = round(data[x]['Delivered']/data[x]['Cap'],2)

# Get data from solution
data_sol = json.load(open('solutions/main_analysis/solution_{date}_regular.json'.format(date=date)))['Ordered_flow']
data_sol = {z:[data_sol[z][x+1] - data_sol[z][x] for x in range(len(data_sol[z])-1) if (data_sol[z][x+1] - data_sol[z][x]) < 0] for z in data_sol}
data_sol = {z:[abs(x) for x in data_sol[z]] for z in data_sol}
data_sol = {z:sum(data_sol[z]) for z in data_sol}
data_sol = {z:{'Cap':capacities[z],
               'Delivered':data_sol[z]} for z in data_sol}
for x in data_sol:
    data_sol[x]['Used_CAP'] = round(data_sol[x]['Delivered']/data_sol[x]['Cap'],2)

data_sol

# data
original_Y = [data[x]['Used_CAP'] for x in data] #data
original_L = [x for x in data] # labels
optimized_Y = [data_sol[x]['Used_CAP'] for x in data_sol] # data
optimized_L = [x for x in data_sol] # labels

# Colors for labels (here, I'm using a colormap to generate colors)
colors = plt.cm.rainbow(np.linspace(0, 1, len(set(original_L + optimized_L))))

# Create a color dictionary
color_dict = dict(zip(set(original_L + optimized_L), colors))

# Assign X = 0.5 for 'Original' points and X = 1.5 for 'Optimized' points
original_X = 0.5 * np.ones(len(original_Y))
optimized_X = 1.5 * np.ones(len(optimized_Y))

# Plotting
fig, ax = plt.subplots()

# For original
for i in range(len(original_Y)):
    ax.scatter(original_X[i], original_Y[i], marker='o', color=color_dict[original_L[i]], edgecolor='black')
    ax.annotate(original_L[i], (original_X[i], original_Y[i]))

# For optimized
for i in range(len(optimized_Y)):
    ax.scatter(optimized_X[i], optimized_Y[i], marker='o', color=color_dict[optimized_L[i]], edgecolor='black')
    ax.annotate(optimized_L[i], (optimized_X[i], optimized_Y[i]))

# Add a red horizontal line at Y=1.0
ax.axhline(y=1.0, color='r')

# Add xticks, grid, and title
ax.set_xticks([0.5, 1.5])
ax.set_xticklabels(['Original', 'Optimized'])
ax.set_xlim([0, 2])  # setting the x-axis limits
ax.set_ylim([0, 2])
ax.set_title('Capacity plot - {date}'.format(date=date))
ax.set_xlabel('Status')  # setting the x-axis label

# Show the plot
plt.tight_layout()  # This will ensure that all labels are visible
plt.savefig('plots/Q1plots/plot_{date}'.format(date=date), dpi=400)
plt.show()

### Q2

In [ ]:
dates = ['2022-01-27', '2022-03-08','2022-03-09','2022-05-03','2022-06-29','2022-07-06','2022-07-26','2022-07-28','2022-09-08','2022-09-29','2022-09-30','2022-10-04','2022-10-14','2022-10-21']
obj_values = {}

for i in dates:
    obj_values[i] = {}
    obj_values[i]['Reg'] = json.load(open('solutions/main_analysis/solution_{date}_{type}.json'.format(date=i, type='regular')))['OBJ']
    obj_values[i]['HigherLimit'] = json.load(open('solutions/main_analysis/solution_{date}_{type}.json'.format(date=i, type='higherlimit')))['OBJ']

obj_values

In [ ]:
# Create lineplot
fig, ax = plt.subplots(figsize=(10,7))

# Plot lines
ax.plot(dates, [obj_values[x]['Reg'] for x in obj_values], label='Regular limit ($6 m^3$)')
ax.plot(dates, [obj_values[x]['HigherLimit'] for x in obj_values], label='Higher limit ($12 m^3$)')
diff = round(sum([obj_values[x]['HigherLimit'] for x in obj_values])-sum([obj_values[x]['Reg'] for x in obj_values]),2)

# Format dates
ax.set_xticks(range(len(dates)))
ax.set_xticklabels(dates)
fig.autofmt_xdate()

# Add titles
ax.set_title('Objective values found over time\n Current limit: 6 $m^3$, Evaluated limit: 12 $m^3$, Δ = {diff} km'.format(diff=diff))
ax.set_xlabel('Date')
ax.set_ylabel('Objective value (distance in km)')

# Add legend
ax.legend(loc='upper center', bbox_to_anchor=(0.5, -0.15), ncol=2, shadow=True)
box = ax.get_position()
ax.set_position([box.x0, box.y0 + box.height * 0.15,
                 box.width, box.height * 0.85])
#plt.legend(loc='upper center', bbox_to_anchor=(0.5, -0.10), ncol=2)
plt.tight_layout()
plt.grid()
plt.savefig('plots/higherlimitobj.png', dpi=400)

In [ ]:
aggregated_data = {'2022-01-27': {'regular': {'Watergeus': {'Cap': 103,
    'Delivered': 59,
    'Used_CAP': 0.57},
   'Waterbuffel': {'Cap': 103, 'Delivered': 27, 'Used_CAP': 0.26},
   'Waterval': {'Cap': 83, 'Delivered': 61, 'Used_CAP': 0.73},
   'Waterman': {'Cap': 83, 'Delivered': 58, 'Used_CAP': 0.7}},
  'higherlimit': {'Watergeus': {'Cap': 103, 'Delivered': 34, 'Used_CAP': 0.33},
   'Waterbuffel': {'Cap': 103, 'Delivered': 59, 'Used_CAP': 0.57},
   'Waterval': {'Cap': 83, 'Delivered': 54, 'Used_CAP': 0.65},
   'Waterman': {'Cap': 83, 'Delivered': 52, 'Used_CAP': 0.63}}},
 '2022-03-08': {'regular': {'Watergeus': {'Cap': 103,
    'Delivered': 111,
    'Used_CAP': 1.08},
   'Waterbuffel': {'Cap': 103, 'Delivered': 97, 'Used_CAP': 0.94},
   'Waterval': {'Cap': 83, 'Delivered': 79, 'Used_CAP': 0.95},
   'Waterman': {'Cap': 83, 'Delivered': 95, 'Used_CAP': 1.14}},
  'higherlimit': {'Watergeus': {'Cap': 103, 'Delivered': 82, 'Used_CAP': 0.8},
   'Waterbuffel': {'Cap': 103, 'Delivered': 134, 'Used_CAP': 1.3},
   'Waterval': {'Cap': 83, 'Delivered': 130, 'Used_CAP': 1.57},
   'Waterman': {'Cap': 83, 'Delivered': 101, 'Used_CAP': 1.22}}},
 '2022-03-09': {'regular': {'Watergeus': {'Cap': 103,
    'Delivered': 61,
    'Used_CAP': 0.59},
   'Waterbuffel': {'Cap': 103, 'Delivered': 66, 'Used_CAP': 0.64},
   'Waterval': {'Cap': 83, 'Delivered': 63, 'Used_CAP': 0.76},
   'Waterman': {'Cap': 83, 'Delivered': 34, 'Used_CAP': 0.41}},
  'higherlimit': {'Watergeus': {'Cap': 103, 'Delivered': 71, 'Used_CAP': 0.69},
   'Waterbuffel': {'Cap': 103, 'Delivered': 71, 'Used_CAP': 0.69},
   'Waterval': {'Cap': 83, 'Delivered': 73, 'Used_CAP': 0.88},
   'Waterman': {'Cap': 83, 'Delivered': 30, 'Used_CAP': 0.36}}},
 '2022-05-03': {'regular': {'Watergeus': {'Cap': 103,
    'Delivered': 98,
    'Used_CAP': 0.95},
   'Waterbuffel': {'Cap': 103, 'Delivered': 86, 'Used_CAP': 0.83},
   'Waterval': {'Cap': 83, 'Delivered': 103, 'Used_CAP': 1.24},
   'Waterman': {'Cap': 83, 'Delivered': 57, 'Used_CAP': 0.69}},
  'higherlimit': {'Watergeus': {'Cap': 103, 'Delivered': 85, 'Used_CAP': 0.83},
   'Waterbuffel': {'Cap': 103, 'Delivered': 119, 'Used_CAP': 1.16},
   'Waterval': {'Cap': 83, 'Delivered': 85, 'Used_CAP': 1.02},
   'Waterman': {'Cap': 83, 'Delivered': 122, 'Used_CAP': 1.47}}},
 '2022-06-29': {'regular': {'Watergeus': {'Cap': 103,
    'Delivered': 59,
    'Used_CAP': 0.57},
   'Waterbuffel': {'Cap': 103, 'Delivered': 63, 'Used_CAP': 0.61},
   'Waterval': {'Cap': 83, 'Delivered': 68, 'Used_CAP': 0.82},
   'Waterman': {'Cap': 83, 'Delivered': 45, 'Used_CAP': 0.54}},
  'higherlimit': {'Watergeus': {'Cap': 103, 'Delivered': 73, 'Used_CAP': 0.71},
   'Waterbuffel': {'Cap': 103, 'Delivered': 73, 'Used_CAP': 0.71},
   'Waterval': {'Cap': 83, 'Delivered': 83, 'Used_CAP': 1.0},
   'Waterman': {'Cap': 83, 'Delivered': 50, 'Used_CAP': 0.6}}},
 '2022-07-06': {'regular': {'Watergeus': {'Cap': 103,
    'Delivered': 71,
    'Used_CAP': 0.69},
   'Waterbuffel': {'Cap': 103, 'Delivered': 70, 'Used_CAP': 0.68},
   'Waterval': {'Cap': 83, 'Delivered': 46, 'Used_CAP': 0.55},
   'Waterman': {'Cap': 83, 'Delivered': 60, 'Used_CAP': 0.72}},
  'higherlimit': {'Watergeus': {'Cap': 103, 'Delivered': 85, 'Used_CAP': 0.83},
   'Waterbuffel': {'Cap': 103, 'Delivered': 88, 'Used_CAP': 0.85},
   'Waterval': {'Cap': 83, 'Delivered': 84, 'Used_CAP': 1.01},
   'Waterman': {'Cap': 83, 'Delivered': 40, 'Used_CAP': 0.48}}},
 '2022-07-26': {'regular': {'Watergeus': {'Cap': 103,
    'Delivered': 118,
    'Used_CAP': 1.15},
   'Waterbuffel': {'Cap': 103, 'Delivered': 101, 'Used_CAP': 0.98},
   'Waterval': {'Cap': 83, 'Delivered': 106, 'Used_CAP': 1.28},
   'Waterman': {'Cap': 83, 'Delivered': 70, 'Used_CAP': 0.84}},
  'higherlimit': {'Watergeus': {'Cap': 103,
    'Delivered': 143,
    'Used_CAP': 1.39},
   'Waterbuffel': {'Cap': 103, 'Delivered': 141, 'Used_CAP': 1.37},
   'Waterval': {'Cap': 83, 'Delivered': 52, 'Used_CAP': 0.63},
   'Waterman': {'Cap': 83, 'Delivered': 140, 'Used_CAP': 1.69}}},
 '2022-07-28': {'regular': {'Watergeus': {'Cap': 103,
    'Delivered': 73,
    'Used_CAP': 0.71},
   'Waterbuffel': {'Cap': 103, 'Delivered': 73, 'Used_CAP': 0.71},
   'Waterval': {'Cap': 83, 'Delivered': 78, 'Used_CAP': 0.94},
   'Waterman': {'Cap': 83, 'Delivered': 35, 'Used_CAP': 0.42}},
  'higherlimit': {'Watergeus': {'Cap': 103, 'Delivered': 87, 'Used_CAP': 0.84},
   'Waterbuffel': {'Cap': 103, 'Delivered': 103, 'Used_CAP': 1.0},
   'Waterval': {'Cap': 83, 'Delivered': 72, 'Used_CAP': 0.87},
   'Waterman': {'Cap': 83, 'Delivered': 106, 'Used_CAP': 1.28}}},
 '2022-09-08': {'regular': {'Watergeus': {'Cap': 103,
    'Delivered': 69,
    'Used_CAP': 0.67},
   'Waterbuffel': {'Cap': 103, 'Delivered': 73, 'Used_CAP': 0.71},
   'Waterval': {'Cap': 83, 'Delivered': 76, 'Used_CAP': 0.92},
   'Waterman': {'Cap': 83, 'Delivered': 36, 'Used_CAP': 0.43}},
  'higherlimit': {'Watergeus': {'Cap': 103, 'Delivered': 89, 'Used_CAP': 0.86},
   'Waterbuffel': {'Cap': 103, 'Delivered': 91, 'Used_CAP': 0.88},
   'Waterval': {'Cap': 83, 'Delivered': 89, 'Used_CAP': 1.07},
   'Waterman': {'Cap': 83, 'Delivered': 35, 'Used_CAP': 0.42}}},
 '2022-09-29': {'regular': {'Watergeus': {'Cap': 103,
    'Delivered': 66,
    'Used_CAP': 0.64},
   'Waterbuffel': {'Cap': 103, 'Delivered': 29, 'Used_CAP': 0.28},
   'Waterval': {'Cap': 83, 'Delivered': 63, 'Used_CAP': 0.76},
   'Waterman': {'Cap': 83, 'Delivered': 63, 'Used_CAP': 0.76}},
  'higherlimit': {'Watergeus': {'Cap': 103, 'Delivered': 42, 'Used_CAP': 0.41},
   'Waterbuffel': {'Cap': 103, 'Delivered': 63, 'Used_CAP': 0.61},
   'Waterval': {'Cap': 83, 'Delivered': 60, 'Used_CAP': 0.72},
   'Waterman': {'Cap': 83, 'Delivered': 64, 'Used_CAP': 0.77}}},
 '2022-09-30': {'regular': {'Watergeus': {'Cap': 103,
    'Delivered': 74,
    'Used_CAP': 0.72},
   'Waterbuffel': {'Cap': 103, 'Delivered': 63, 'Used_CAP': 0.61},
   'Waterval': {'Cap': 83, 'Delivered': 53, 'Used_CAP': 0.64},
   'Waterman': {'Cap': 83, 'Delivered': 78, 'Used_CAP': 0.94}},
  'higherlimit': {'Watergeus': {'Cap': 103,
    'Delivered': 108,
    'Used_CAP': 1.05},
   'Waterbuffel': {'Cap': 103, 'Delivered': 49, 'Used_CAP': 0.48},
   'Waterval': {'Cap': 83, 'Delivered': 108, 'Used_CAP': 1.3},
   'Waterman': {'Cap': 83, 'Delivered': 99, 'Used_CAP': 1.19}}},
 '2022-10-04': {'regular': {'Watergeus': {'Cap': 103,
    'Delivered': 82,
    'Used_CAP': 0.8},
   'Waterbuffel': {'Cap': 103, 'Delivered': 89, 'Used_CAP': 0.86},
   'Waterval': {'Cap': 83, 'Delivered': 66, 'Used_CAP': 0.8},
   'Waterman': {'Cap': 83, 'Delivered': 96, 'Used_CAP': 1.16}},
  'higherlimit': {'Watergeus': {'Cap': 103, 'Delivered': 99, 'Used_CAP': 0.96},
   'Waterbuffel': {'Cap': 103, 'Delivered': 109, 'Used_CAP': 1.06},
   'Waterval': {'Cap': 83, 'Delivered': 78, 'Used_CAP': 0.94},
   'Waterman': {'Cap': 83, 'Delivered': 83, 'Used_CAP': 1.0}}},
 '2022-10-14': {'regular': {'Watergeus': {'Cap': 103,
    'Delivered': 67,
    'Used_CAP': 0.65},
   'Waterbuffel': {'Cap': 103, 'Delivered': 32, 'Used_CAP': 0.31},
   'Waterval': {'Cap': 83, 'Delivered': 64, 'Used_CAP': 0.77},
   'Waterman': {'Cap': 83, 'Delivered': 60, 'Used_CAP': 0.72}},
  'higherlimit': {'Watergeus': {'Cap': 103, 'Delivered': 63, 'Used_CAP': 0.61},
   'Waterbuffel': {'Cap': 103, 'Delivered': 64, 'Used_CAP': 0.62},
   'Waterval': {'Cap': 83, 'Delivered': 28, 'Used_CAP': 0.34},
   'Waterman': {'Cap': 83, 'Delivered': 58, 'Used_CAP': 0.7}}},
 '2022-10-21': {'regular': {'Watergeus': {'Cap': 103,
    'Delivered': 61,
    'Used_CAP': 0.59},
   'Waterbuffel': {'Cap': 103, 'Delivered': 55, 'Used_CAP': 0.53},
   'Waterval': {'Cap': 83, 'Delivered': 61, 'Used_CAP': 0.73},
   'Waterman': {'Cap': 83, 'Delivered': 60, 'Used_CAP': 0.72}},
  'higherlimit': {'Watergeus': {'Cap': 103, 'Delivered': 43, 'Used_CAP': 0.42},
   'Waterbuffel': {'Cap': 103, 'Delivered': 80, 'Used_CAP': 0.78},
   'Waterval': {'Cap': 83, 'Delivered': 74, 'Used_CAP': 0.89},
   'Waterman': {'Cap': 83, 'Delivered': 73, 'Used_CAP': 0.88}}}}

In [ ]:
plot_data = {'regular': [0, 2, 0, 1, 0, 0, 2, 0, 0, 0, 0, 1, 0, 0],
 'higherlimit': [0, 3, 0, 3, 0, 1, 3, 1, 1, 0, 3, 1, 0, 0]}

sum(plot_data['regular'])
sum(plot_data['higherlimit'])

In [ ]:
# Original
evides = pd.read_csv('data/evides_withAIS.csv')
evides = evides[evides['Scheepstype'] != 'Passagiersschip, opleidingschip']
evides['Outside area'] = [location_in_area([evides['Latitude'].loc[x],evides['Longitude'].loc[x]], ((51.86, 3.9), (52,4.6))) for x in evides.index]
evides = evides[evides['Outside area']==1]

dates = list(set(evides['Datum']))
waterq_original = [sum(evides['Hoeveelheid (m3)'][evides['Datum']==x]) for x in dates]
watern_original = [len(evides['Hoeveelheid (m3)'][evides['Datum']==x]) for x in dates]

# Higher limit
evides = pd.read_csv('data/evides_higherlimit.csv')
evides = evides[evides['Scheepstype'] != 'Passagiersschip, opleidingschip']
evides['Outside area'] = [location_in_area([evides['Latitude'].loc[x],evides['Longitude'].loc[x]], ((51.86, 3.9), (52,4.6))) for x in evides.index]
evides = evides[evides['Outside area']==1]

dates = list(set(evides['Datum']))
waterq_higherlim = [sum(evides['Hoeveelheid (m3)'][evides['Datum']==x]) for x in dates]
watern_higherlim = [len(evides['Hoeveelheid (m3)'][evides['Datum']==x]) for x in dates]

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from statistics import mean

# Create scatterplot
fig, ax = plt.subplots(figsize=(10, 7))

# Plot scatter points
ax.scatter(dates, waterq_original, label='Regular limit ($6 m^3$)')
ax.scatter(dates, waterq_higherlim, label='Higher limit ($12 m^3$)')

# Format dates
ax.set_xticks(np.arange(0, len(dates), 7))  # Set ticks every 7 days
ax.set_xticklabels(dates[::7])  # Display labels every 7 days

# Add titles
ax.set_title('Sum of cubic metres of $m^3$ of water delivered per day - 6 $m^3$ vs. 12 $m^3$ shipment limit  \n Mean amount (6 $m^3$): {avg1} $m^3$, Mean amount (12 $m^3$): {avg2} $m^3$'.format(
    avg1=round(mean(waterq_original),2),
    avg2=round(mean(waterq_higherlim),2)))
ax.set_xlabel('Week')
ax.set_ylabel('Cubic metres of $m^3$ of water delivered')
fig.autofmt_xdate()

# Add legend
ax.legend(loc='upper center', bbox_to_anchor=(0.5, -0.15), ncol=2, shadow=True)
box = ax.get_position()
ax.set_position([box.x0, box.y0 + box.height * 0.15, box.width, box.height * 0.85])

plt.tight_layout()
plt.grid()
#plt.savefig('plots/numberofm3.png', dpi=400)

In [ ]:
# Create scatterplot
fig, ax = plt.subplots(figsize=(10, 7))

# Plot scatter points
ax.scatter(dates, watern_original, label='Regular limit ($6 m^3$)')
ax.scatter(dates, watern_higherlim, label='Higher limit ($12 m^3$)')

# Format dates
ax.set_xticks(np.arange(0, len(dates), 7))  # Set ticks every 7 days
ax.set_xticklabels(dates[::7])  # Display labels every 7 days

# Add titles
ax.set_title('Number of shipments per day - 6 $m^3$ vs. 12 $m^3$ shipment limit  \n Mean amount (6 $m^3$): {avg1}, Mean amount (12 $m^3$): {avg2}'.format(
    avg1=round(mean(watern_original),0),
    avg2=round(mean(watern_higherlim),0)))
ax.set_xlabel('Week')
ax.set_ylabel('Number of shipments')
fig.autofmt_xdate()

# Add legend
ax.legend(loc='upper center', bbox_to_anchor=(0.5, -0.15), ncol=2, shadow=True)
box = ax.get_position()
ax.set_position([box.x0, box.y0 + box.height * 0.15, box.width, box.height * 0.85])

plt.tight_layout()
plt.grid()
#plt.savefig('plots/numberofm3.png', dpi=400)